# Modelo para un Sistema de Recomendación

Vamos a realizar un modelo basado en un Sitema de Recomendación, cuyo objetivo es aconsejar a un potencial usuario una lista de películas ordenadas según la preferencia estimada, en función de las valoraciones que este ha dado a otras películas que haya visto y las valoraciones que han dado el resto de usuarios. Este modelo va a tener un enfoque de inteligencia colectiva, es decir, va a tener en cuenta las valoraciones de todos los usuarios, tanto aquellos que tienen gustos de cine similares al usuario que se quiere recomendar, como aquellos cuyas preferencias son desemejantes. Para ello se han tomado los datasets de la plataforma *MovieLens*, que se encuentran en el siguiente enlace:  

https://grouplens.org/datasets/movielens/latest/

Contiene datos de valoraciones de usuarios a películas desde el 29 de marzo de 1996 al 24 de septiembre de 2018. Se seleccionaron aleatoriamente valoraciones de usuarios que hubiesen realizado al menos 20 valoraciones.

## Carga de las librerías necesarias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

## Exploración de los datos

In [2]:
ratings = pd.read_csv('data/ratings.csv')
movies = pd.read_csv('data/movies.csv')

In [3]:
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

En el dataset `ratings` tenemos almacenadas un total de 100836 valoraciones (entre 0 y 5) de los usuarios a las películas. Tanto los usuarios como las películas están representadas por un id. Contamos también con una variable `timestamp` que indica la fecha en la que se realizó la valoración por el usuario. Eliminaremos esta columna del dataset pues no es relevante para el estudio.

In [5]:
ratings = ratings.drop('timestamp', axis = 1) if 'timestamp' in ratings.columns  else ratings

Veamos un breve análisis descriptivo para obtener una visión general de cuánto y cómo valoran los usuarios.

In [6]:
df = ratings.groupby('userId').agg(media_reseñas= ('rating','mean'))
df['num_reseñas'] = ratings.groupby('userId').agg(num_reseñas=('rating', 'count'))
df.sort_values('num_reseñas', ascending = False)

,media_reseñas,num_reseñas
userId,,
414,3.391957,2698
599,2.642050,2478
474,3.398956,2108
448,2.847371,1864
274,3.235884,1346
...,...,...
442,1.275000,20
569,4.000000,20
320,3.525000,20


En efecto los usuarios han realizado al menos 20 reseñas. Hay usuarios que han realizado un gran número de ellas, habiendo realizado 2698 el que más.

In [7]:
df.sort_values('media_reseñas', ascending = False)

,media_reseñas,num_reseñas
userId,,
53,5.000000,20
251,4.869565,23
515,4.846154,26
25,4.807692,26
30,4.735294,34
...,...,...
567,2.245455,385
153,2.217877,179
508,2.145833,24


Teniendo en cuenta que el número de reseñas es al menos 20 y observando el dataframe anterior, observamos la hetereogeneidad en la dureza de las reseñas, pues hay gente con reseñas muy altas y otros que dan valoraciones bastante bajas.

Pasemos ahora al segundo dataset, que contiene la informaciíon sobre las películas

In [8]:
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


En el dataset `movies` tenemos la información sobre las películas valoradas por los usuarios, en total 9742, contando con el título, en el que se encuentra la fecha de estreno, y una variable que contiene los géneros en los que se puede clasificar la película correspondiente.

In [9]:
print(f"El número de películas es {len(movies.movieId.unique())}")
print(f"El número de películas valoradas por los usuarios son {len(ratings.movieId.unique())}")

El número de películas es 9742
El número de películas valoradas por los usuarios son 9724


Hay por tanto películas que no tienen ninguna valoración. Vamos a eliminar esas películas del estudio, pues no es razonable recomendar una película que no tiene valoraciones. Además podemos confirmar que el id de la variable `movieId` es único.

In [10]:
movies.sort_values('movieId', ascending = False)

,movieId,title,genres
9741,193609,Andrew Dice Clay: Dice Rules (1991),Comedy
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation
9739,193585,Flint (2017),Drama
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
...,...,...,...
4,5,Father of the Bride Part II (1995),Comedy
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
2,3,Grumpier Old Men (1995),Comedy|Romance
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [11]:
ids =  ratings.movieId.unique()
index = [ x in ids for x in movies.movieId]
movies = movies.loc[index]

Procedamos a construir el dataframe de pares película-usuario, en la que almacenamos las valoraciones de los usuarios a las distintas películas.  valoración nula.

In [12]:
users_all = ratings.userId.unique()
movies_all = movies.movieId
X = pd.DataFrame(np.zeros((len(movies_all),len(users_all))), columns = users_all, index = movies_all)-1.0
X = pd.DataFrame(np.nan, columns = users_all, index = movies_all)
for m,u,r in zip(ratings.movieId, ratings.userId, ratings.rating):
    X.loc[m,u] = r
X

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creación del modelo

Para el desarrollo del modelo vamos a hacer uso de la similitud del coseno. Dados dos vectores $a = \left(a_1,\dots, a_n\right)$, $b = \left(b_1,\dots, b_n\right)$, se define como: $$ s(a,b) = \frac{\sum_{i=1}^n a_ib_i}{\sqrt{\sum_{i=1}^n a_i^2}\sqrt{\sum_{i=1}^n b_i^2}}   $$

Escogemos esta métrica de la similitud pues es consistente ante distintas magnitudes de los vectores, pues tan solo va a influir el ángulo que forman. Esto es razonable pues como habíamos visto anteriormente, hay usuarios con gustos similares pero con distinta severidad a la hora de puntuar. En cuanto a los valores nulos, habría varias opciones: calcular la similitud entre dos usuarios teniendo en cuenta únicamente las películas que ambos han visto, imputar los valores nulos con la media de las valoraciones de las películas vistas por el usuario o hacerlo con la media de las valoraciones de la película en cuestión para los demás usuarios. 

La primera de las opciones la descartamos porque computacionalmente no es nada eficiente, pues habría que comprobar para cada par de usuario las películas valoradas coincidentes, lo que aumenta mucho la complejidad del algoritmo. Empecemos adoptando el segundo método.

In [13]:
def recomendar_peliculas(X, usuario_id, n=3):
    X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    
    similitud_usuarios = cosine_similarity(X_filled.T)
    idx_usuario = X.columns.get_loc(usuario_id)
    similitudes = similitud_usuarios[idx_usuario]

    peliculas_no_vistas = X[usuario_id].isna()
    if peliculas_no_vistas.sum() == 0:
        return "El usuario ha visto todas las películas."
    
    valoraciones_estimadas = (X_filled * similitudes).sum(axis=1) / np.abs(similitudes).sum()
    
    recomendaciones = valoraciones_estimadas.loc[peliculas_no_vistas].sort_values(ascending=False).head(n)  
    print(f"Se muestran a continuación las películas recomendadas para el usuario {usuario_id}: \n")
    for p,s in dict(recomendaciones).items():
       print(f"{movies.loc[movies['movieId'] == p].iloc[0]['title']}: {s} \n ")  
    
    return recomendaciones.head(n)

In [14]:
usuario_id = 9
recomendaciones = recomendar_peliculas(X, usuario_id)

Se muestran a continuación las películas recomendadas para el usuario 9: 

Shawshank Redemption, The (1994): 4.0361154061478155 
 
Forrest Gump (1994): 3.935764237148868 
 
Pulp Fiction (1994): 3.932253199409871 
 


En caso de que consideremos los usuarios fijos, podemos hacer más eficiente el modelo calculando previamente la matriz de similitudes, de manera que para el modelo no sea necesario el cálculo de esta matriz cada vez que se requiera recomendar a un usuario distinto.

In [15]:
def calcula_similitudes(X):
    X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    similitud_usuarios = cosine_similarity(X_filled.T)
    return similitud_usuarios

In [16]:
def recomendar_peliculas(X,usuario_id, mat, n=3):
    X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    idx_usuario = X.columns.get_loc(usuario_id)
    similitudes = mat[idx_usuario]

    peliculas_no_vistas = X[usuario_id].isna()
    if peliculas_no_vistas.sum() == 0:
        return "El usuario ha visto todas las películas."
    
    valoraciones_estimadas = (X_filled * similitudes).sum(axis=1) / np.abs(similitudes).sum()
    
    recomendaciones = valoraciones_estimadas[peliculas_no_vistas].sort_values(ascending=False).head(n)  
    print(f"Se muestran a continuación las películas recomendadas para el usuario {usuario_id}: \n")
    for p,s in dict(recomendaciones).items():
        print(f"{movies.loc[movies['movieId'] == p].iloc[0]['title']}: {s} \n ")   
    
    return recomendaciones

In [17]:
mat = calcula_similitudes(X)
usuarios_id = [n for n in range(10,15)]
for usuario_id in usuarios_id:
    recomendar_peliculas(X, usuario_id,mat)
    print('---------------------------------------------------- \n')

Se muestran a continuación las películas recomendadas para el usuario 10: 

Shawshank Redemption, The (1994): 4.036113354998536 
 
Star Wars: Episode IV - A New Hope (1977): 3.8920272879745164 
 
Silence of the Lambs, The (1991): 3.887848577809959 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 11: 

Pulp Fiction (1994): 3.932253709769485 
 
Star Wars: Episode IV - A New Hope (1977): 3.8920330999707997 
 
Matrix, The (1999): 3.8736521601789544 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 12: 

Shawshank Redemption, The (1994): 4.036116751498912 
 
Forrest Gump (1994): 3.935765752876898 
 
Pulp Fiction (1994): 3.9322510075595494 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 13: 

Shawshank Redemption, The (1994): 4.036117322660477 
 
Forrest Gump

Demos ahora la alternativa en el modelo de implementar la tercera opción descrita en el modelo, es decir, imputando los valores nulos por la media de las valoraciones del resto de usuarios a la película correspondiente.

In [18]:
def calcula_similitudes(X, fillasrows = False):
    if fillasrows:
        X_filled = X.apply(lambda row: row.fillna(row.mean()), axis=1)
    else:
        X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    similitud_usuarios = cosine_similarity(X_filled.T)
    return similitud_usuarios

In [19]:
def recomendar_peliculas(X,usuario_id, mat, n=3, fillasrows = False):
    if fillasrows:
        X_filled = X.apply(lambda row: row.fillna(row.mean()), axis=1)
    else:
        X_filled = X.apply(lambda col: col.fillna(col.mean()), axis=0)
    idx_usuario = X.columns.get_loc(usuario_id)
    similitudes = mat[idx_usuario]

    peliculas_no_vistas = X[usuario_id].isna()
    if peliculas_no_vistas.sum() == 0:
        return "El usuario ha visto todas las películas."
    
    valoraciones_estimadas = (X_filled * similitudes).sum(axis=1) / np.abs(similitudes).sum()
    
    recomendaciones = valoraciones_estimadas[peliculas_no_vistas].sort_values(ascending=False).head(n)  
    print(f"Se muestran a continuación las películas recomendadas para el usuario {usuario_id}: \n")
    for p,s in dict(recomendaciones).items():
       print(f"{movies.loc[movies['movieId'] == p].iloc[0]['title']}: {s} \n ")   
    
    return recomendaciones

In [20]:
mat = calcula_similitudes(X, fillasrows = True)
usuarios_id = [n for n in range(10,15)]
for usuario_id in usuarios_id:
    recomendaciones = recomendar_peliculas(X, usuario_id,mat, fillasrows = True)
    print('---------------------------------------------------- \n')

Se muestran a continuación las películas recomendadas para el usuario 10: 

What Love Is (2007): 5.0 
 
What Men Talk About (2010): 5.0 
 
The Jinx: The Life and Deaths of Robert Durst (2015): 5.0 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 11: 

A Perfect Day (2015): 5.0 
 
Trailer Park Boys (1999): 5.0 
 
Animals are Beautiful People (1974): 5.0 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 12: 

The Adventures of Sherlock Holmes and Doctor Watson: King of Blackmailers (1980): 4.999999999999999 
 
George Carlin: Back in Town (1996): 4.999999999999999 
 
12 Chairs (1976): 4.999999999999999 
 
---------------------------------------------------- 

Se muestran a continuación las películas recomendadas para el usuario 13: 

Breed, The (2006): 5.0 
 
Scooby-Doo! and the Loch Ness Monster (2004): 5.0 
 
Ugly Duckling and Me!, 

Vemos una clara sobreestimación de las valoraciones para los usuarios, y es que este último método es en un principio es razonable pues pese a que la valoración es algo subjetivo, se puede suponer que las valoraciones de los usuarios no difieren en demasía de la media. Sin embargo, de esta manera las películas que no hayan visto ambos usuarios van a ser calificadas de la misma forma, produciendo esto una sobreestimación en la similitud. Por ello, es más conveniente la segunda opción, es decir, sustituir por la media de las valoraciones de las películas para el usuario, ya que de esta manera se tiene en cuenta la diferencia de la dureza al valorar de los distintos usuarios

Las opciones posibles que tendríamos en principio para una mejora sobre el modelo residen principalmente en la forma de tratar los valores nulos del datafrmae $X$ de pares película-usuario, en la manera de calcular la similitud entre usuarios, y la ponderación a la hora de calcular las predicciones de las valoraciones. De momento, la mejor técnica es la sustitución de los valores nulos por la media de las valoraciones del usuario. 

Podemos afinar más si tenemos en cuenta que la valoración de un usuario va a estar estrechamente relacionada con el género de la película en cuestión. Por ello, podríamos sustituir los valores nulos por la valoración media del usuario para las películas de ese mismo género en caso de haber valorado películas del género en cuestión, y sustituir por la media de todas las películas en caso contrario. También podemos emplear otros métodos como la imputación por KNN, tomando como valoración de la película la media de los usuarios más cercanos que han valorado esa película. Sin embargo, estos métodos son muy costosos computacionalmente y no es posible con cantidades de datos importantes.

### Citación

F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. https://doi.org/10.1145/2827872